<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/Demo_pyswisseph_v_2_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---

Website - [Parashar21](https://parashar21.blogspot.com)

#Install Swiss Ephemeris <br>
## This is a demonstration of the usage of the Swiss Ephemeris Software in python on the Google Colab Platform
https://pypi.org/project/pyswisseph/  - Python Package Documentation <br>
https://www.astro.com/swisseph/swephinfo_e.htm <br>
https://www.astro.com/swisseph/swephprg.htm - Programmer Documentation <br>
https://pythonhosted.org/pyswisseph/swisseph-module.html

In [1]:
!pip install pyswisseph > /dev/null

In [2]:
!wget -q http://www.astro.com/ftp/swisseph/ephe/seas_18.se1
!wget -q http://www.astro.com/ftp/swisseph/ephe/semo_18.se1
!wget -q http://www.astro.com/ftp/swisseph/ephe/sepl_18.se1

!mkdir ephe
!mv *.se1 ephe

In [2]:
import swisseph as swe
swe.version

'2.10.03'

In [3]:
#Constants and Flags necessary for this application are set here
#
swe.set_ephe_path('/content/ephe') # set path to ephemeris files
# Calendar : Julian or Gregorian
SE_GREG_CAL = 1
gregflag = SE_GREG_CAL
#
# Ayanamsha type : Lahiri = 1
swe.set_sid_mode(1)  # Lahiri Aynamsha
#
# whether speed will be calculated along with position of planet
SEFLG_SPEED = int(256)
iflag = SEFLG_SPEED
#hsys = P, Placidus
#ascii P = 080
hsysP =  bytes('P', 'utf-8')

#Personal Data
## Data Input

In [4]:
# Date & Time of Birth
# Convert UTC date to Julian Date
JD = swe.utc_to_jd(1950,9,17,11,0,0,gregflag)
print (JD[0], JD[1])
# JD[0] is Ephemeris Time
# JD[1] is Universal Time 
natalUT = JD[1]
print(natalUT, 'Julian Date in Universal Time')
natalLON = 72.36
natalLAT = 23.48

2433541.9586713817 2433541.9583333335
2433541.9583333335 Julian Date in Universal Time


#Calculations

In [5]:
#Ayanamsha
ayanamsha = swe.get_ayanamsa(natalUT)
print('Lahiri Ayanamsha :', ayanamsha)

Lahiri Ayanamsha : 23.168645873129833


In [6]:
#Position of Planets without SETTING SEFLG_SIDEREAL
#
# body 0 = Sun, 1 = Moon, ... 11 = True Node (Rahu)
bLon = []
bRet = []
for body in [0,1,2,3,4,5,6,11]:
    pData = swe.calc_ut(natalUT,body,iflag)
    #print(pData[0])                    --- this is a major change
    #print(pData[1])                    --- this is a major change
    bLon.append(pData[0][0])            # note this correction
    if pData[0][3] >= 0:                # note this correction
        bRet.append('False')
    else :
        bRet.append('True')
for ix in range(len(bLon)) : print(bLon[ix], bRet[ix])

173.98413896945715 False
245.2670509720432 False
173.7141051466669 True
159.139868694586 False
234.25420923183867 False
329.7386375337111 True
172.84877628380983 False
358.3806098705857 False


In [7]:
#House Position and Ascendants without setting SEFLG_SIDEREAL
# the 13th value is the longitude of Ascendant
#
P = swe.houses(natalUT,natalLAT,natalLON,hsysP)
for ix in range(len(P[0])) : print(P[0][ix])
for ix in range(len(P[1])) : print(P[1][ix])

313.1264431359498
351.49953207220244
26.599492654822736
55.50858475863316
80.45170776055932
104.94438693593315
133.12644313594978
171.49953207220244
206.59949265482274
235.50858475863316
260.4517077605593
284.94438693593315
313.1264431359498
235.50858475863316
233.17025246160762
160.03803418828804
320.77462309696983
326.54430394303506
253.1360224252382
146.5443039430351


In [8]:
#Position of Planetsd using SEFLG_SIDEREAL flag
#
# body 0 = Sun, 1 = Moon, ... 11 = True Node (Rahu)
bLonS = []
bRetS = []
print(iflag)
SEFLG_SIDEREAL = int(64*1024)      #// sidereal positions
iflag |= SEFLG_SIDEREAL
print(iflag)
for body in [0,1,2,3,4,5,6,11]:
    pData = swe.calc_ut(natalUT,body,iflag)
    #print(pData[0])                    --- this is a major change
    #print(pData[1])                    --- this is a major change
    bLonS.append(pData[0][0])            # note this correction
    if pData[0][3] >= 0:                # note this correction
        bRetS.append('False')
    else :
        bRetS.append('True')
print("Change in Longitude for each planet")
for ix in range(len(bLonS)) : print(bLonS[ix]-bLon[ix], bRetS[ix])

256
65792
Change in Longitude for each planet
-23.168757318504134 False
-23.168757318504163 False
-23.168757318504134 True
-23.168757318504134 False
-23.16875731850422 False
-23.168757318504106 True
-23.168757318504134 False
-23.168757318504163 False


In [16]:
print(natalUT,iflag,natalLAT, natalLON,hsysP)

2433541.9583333335 65792 23.48 72.36 b'P'


In [38]:
#House Position and Ascendants without setting SEFLG_SIDEREAL
# the 13th value is the longitude of Ascendant
#
hsysPb = hsysP.decode("utf-8")
print(hsysPb)
PS = swe.houses_ex(natalUT,natalLAT,natalLON,hsysP,iflag)   # the documentation is WRONG, iflag must be the last parameter, not the second
for ix in range(len(PS[0])) : print(PS[0][ix])
for ix in range(len(PS[1])) : print(PS[1][ix])
for ix in range(len(PS[1])) : print(PS[1][0]-P[1][0])

P
289.9576858174456
328.3307747536983
3.4307353363185946
32.33982744012902
57.282950442055174
81.77562961742902
109.95768581744565
148.3307747536983
183.4307353363186
212.33982744012903
237.28295044205518
261.775629617429
289.9576858174456
212.33982744012903
233.17025246160762
136.8692768697839
297.60586577846567
303.3755466245309
229.96726510673406
123.37554662453095
-23.168757318504163
-23.168757318504163
-23.168757318504163
-23.168757318504163
-23.168757318504163
-23.168757318504163
-23.168757318504163
-23.168757318504163


In [39]:
!date

Fri Oct  7 03:13:20 UTC 2022


The output generated here matches with the output generated by the Windows program swetest

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)